# Training a classifier for weights of author vision components

The features are the distance of the author to any tweet in the conversation
indicated by the following structures:
- subtree to viewed tweet from a tweet the author wrote
- root closeness of viewed tweet
- time delta to viewed tweet from tweets the author wrote


In [157]:
import pandas as pd

df = pd.read_pickle("data/vision_graph_data.pkl")

df.describe()


,reply_distance_2,reply_distance_3,reply_distance_4,timedelta,root_distance_0,root_distance_1,root_distance_2,root_distance_3,y,current,beam_node,has_followed_path,has_follow_path
count,3.157800e+06,3.157800e+06,3.157800e+06,3.157800e+06,3.157800e+06,3.157800e+06,3.157800e+06,3.157800e+06,3.157800e+06,3.157800e+06,3.157800e+06,3157800.0,3157800.0
mean,5.017101e-03,3.026791e-03,2.009310e-03,3.850014e+04,3.895750e-02,8.457949e-01,4.832637e-02,2.347457e-02,5.293559e-02,1.395980e+18,1.395910e+18,0.0,0.0
std,7.065360e-02,5.493296e-02,4.478028e-02,1.040748e+06,1.934937e-01,3.611453e-01,2.144550e-01,1.514052e-01,2.239050e-01,4.266853e+17,4.265792e+17,0.0,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.880000e+02,3.880000e+02,0.0,0.0
25%,0.000000e+00,0.000000e+00,0.000000e+00,2.324000e+03,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.526622e+18,1.526541e+18,0.0,0.0
50%,0.000000e+00,0.000000e+00,0.000000e+00,9.048000e+03,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.532639e+18,1.532475e+18,0.0,0.0
75%,0.000000e+00,0.000000e+00,0.000000e+00,3.129100e+04,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.539539e+18,1.539503e+18,0.0,0.0
max,1.000000e+00,1.000000e+00,1.000000e+00,2.148715e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.552260e+18,1.552259e+18,0.0,0.0


In [158]:

# remove non-features
current = df.current
beam_node = df.beam_node
platform = df.platform

platform.value_counts()

twitter    2889148
reddit      268337
delab          315
Name: platform, dtype: int64

In [159]:
df = df[df["platform"] == "twitter"]
# df = df[df["root_distance_0"] == 0]

df = df.drop(["current", "beam_node", "platform", "has_followed_path", "has_follow_path"], axis=1)

# normalize timedelta (put between 0 and 1)
dt = df.timedelta
timedelta_normalized = (dt - dt.min()) / (dt.max() - dt.min())
df.timedelta = timedelta_normalized
df

,reply_distance_2,reply_distance_3,reply_distance_4,timedelta,root_distance_0,root_distance_1,root_distance_2,root_distance_3,y
0,0,0,0,6.114817e-05,1,0,0,0,1
1,0,0,0,5.165878e-07,0,1,0,0,0
2,0,0,0,1.903463e-06,0,1,0,0,0
3,0,0,0,6.599292e-06,0,1,0,0,0
4,0,0,0,1.188617e-05,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
3157795,0,0,0,1.912771e-06,0,1,0,0,0
3157796,0,0,0,2.452628e-06,0,1,0,0,0
3157797,0,0,0,1.433415e-06,1,0,0,0,1
3157798,0,0,0,5.352035e-07,0,1,0,0,0


In [160]:
from sklearn.model_selection import train_test_split

# selecting train and test datasets
train, test = train_test_split(df, test_size=0.2)
train.describe()


,reply_distance_2,reply_distance_3,reply_distance_4,timedelta,root_distance_0,root_distance_1,root_distance_2,root_distance_3,y
count,2.311318e+06,2.311318e+06,2.311318e+06,2.311318e+06,2.311318e+06,2.311318e+06,2.311318e+06,2.311318e+06,2.311318e+06
mean,3.042853e-03,1.868198e-03,1.294500e-03,1.552376e-04,3.847026e-02,8.854636e-01,3.169014e-02,1.394269e-02,4.787788e-02
std,5.507808e-02,4.318227e-02,3.595587e-02,4.860642e-03,1.923287e-01,3.184617e-01,1.751739e-01,1.172532e-01,2.135079e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,1.018283e-05,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,3.930255e-05,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,1.395020e-04,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [161]:
# train the model
y = train.y
x = train.drop("y", axis=1)
print(x.shape)
print(y.shape)


(2311318, 8)
(2311318,)


In [162]:
# convert train dataset to numpy arrays not sure this is neded
#x = x.to_numpy()
#y = y.to_numpy()
#print(x[:5])
#print(y[:5])

In [163]:
# import tensorflow and train the model
import tensorflow as tf

print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

input_shape = (x.shape[1],)
model = Sequential([
    Dense(1, activation='sigmoid', input_shape=input_shape)
])

# stachistical gradient descend as a classifier seem appropriate
model.compile(
    optimizer='sgd',
    loss='binary_crossentropy',
    metrics=['accuracy', 'mae']
)

model.fit(x, y, epochs=3)


2.6.0
Epoch 1/3
72229/72229 [==============================] - 106s 1ms/step - loss: 0.0679 - accuracy: 0.9831 - mae: 0.0380

In [164]:
# evaluate the model on the test set
test_y = test.y
test_x = test.drop("y", axis=1)

In [165]:
# convert train dataset to numpy arrays not sure this is neded
# test_x = test_x.to_numpy()
# test_y = test_y.to_numpy()

In [166]:
loss, accuracy, mae = model.evaluate(test_x, test_y)
print("the accuracy on the training set is {} and the mae is {}".format(accuracy, mae))

18058/18058 [==============================] - 34s 2ms/step - loss: 0.0562 - accuracy: 0.9869 - mae: 0.0245

In [167]:
# have a look at some prediction
reply_distance_2 = test[test["reply_distance_2"] == 1]
reply_distance_2 = reply_distance_2.drop("y", axis=1)
first_rows = reply_distance_2.head(2)
print(first_rows)
model.predict(first_rows)

        reply_distance_2  reply_distance_3  reply_distance_4  timedelta  \
900139                 1                 0                 0   0.000197   
695903                 1                 0                 0   0.000037   

        root_distance_0  root_distance_1  root_distance_2  root_distance_3  
900139                1                0                0                0  
695903                1                0                0                0  


array([[0.8454637],
       [0.8454683]], dtype=float32)

In [168]:
# let's have a look at the weights and biases of the hidden layer

first_layer_weights = model.layers[0].get_weights()[0]
first_layer_biases  = model.layers[0].get_weights()[1]
print(first_layer_weights)
column_names = x.columns.values
for i in range(len(column_names)):
    print("feature {} has weight {} \n" .format(column_names[i], first_layer_weights[i]))



[[-1.1888628 ]
 [ 1.213496  ]
 [-0.6482789 ]
 [-0.21612737]
 [ 4.621445  ]
 [-3.6116598 ]
 [-1.2800721 ]
 [-0.6761108 ]]
feature reply_distance_2 has weight [-1.1888628] 

feature reply_distance_3 has weight [1.213496] 

feature reply_distance_4 has weight [-0.6482789] 

feature timedelta has weight [-0.21612737] 

feature root_distance_0 has weight [4.621445] 

feature root_distance_1 has weight [-3.6116598] 

feature root_distance_2 has weight [-1.2800721] 

feature root_distance_3 has weight [-0.6761108] 

